# Introduction to _explorationlib_ - Lab

This lab has a few goals:

- to get familiar with running that code online using Google's colaboratory.
- to get you familiar with the python library used throughout to simulate and explore exploration.
- to introduce you to Levy walk and Brownian motion agents.


## Section - Info About Colab & Notebooks

Source: Google's introductary notebooks

### Getting started

The document you are reading is not a static web page, but an interactive environment called a **Colab notebook** that lets you write and execute code.  Colab notebooks are Jupyter notebooks that are hosted by Colab. To learn more about the Jupyter project, see [jupyter.org](https://www.jupyter.org).

### Creating and opening notebooks

When you create your own Colab notebooks, they are stored in your Google Drive account.

Notebooks have the file extension "ipynb".  They cannot be opened by normal text editors or IDEs, rather they are used by a notebook server. Here, Google is providing the server, but you can host your own jupyter server on your own computer as well.

The "File" tab can be used to upload or download notebook files, or the create new notebooks.

### Cells
A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

### Code cells
Below is a **code cell**. Once the toolbar button indicates CONNECTED, click in the cell to select it and execute the contents in the following ways:

* Click the **Play icon** in the left gutter of the cell;
* Type **Cmd/Ctrl+Enter** to run the cell in place;
* Type **Shift+Enter** to run the cell and move focus to the next cell (adding one if none exists); or
* Type **Alt+Enter** to run the cell and insert a new code cell immediately below it.

There are additional options for running some or all cells in the **Runtime** menu.


In [ ]:
a = 10
a

### Text cells
This is a **text cell**. You can **double-click** to edit this cell. Text cells
use markdown syntax. To learn more, see the [markdown
guide](/notebooks/markdown_guide.ipynb).

You can also add math to text cells using [LaTeX](http://www.latex-project.org/)
to be rendered by [MathJax](https://www.mathjax.org). Just place the statement
within a pair of **\$** signs. For example `$\sqrt{3x-1}+(1+x)^2$` becomes
$\sqrt{3x-1}+(1+x)^2.$


### Adding and moving cells
You can add new cells by using the **+ CODE** and **+ TEXT** buttons that show when you hover between cells. These buttons are also in the toolbar above the notebook where they can be used to add a cell below the currently selected cell.

You can move a cell by selecting it and clicking **Cell Up** or **Cell Down** in the top toolbar. 

Consecutive cells can be selected by "lasso selection" by dragging from outside one cell and through the group.  Non-adjacent cells can be selected concurrently by clicking one and then holding down Ctrl while clicking another.  Similarly, using Shift instead of Ctrl will select all intermediate cells.

### Integration with Drive

Colaboratory is integrated with Google Drive. It allows you to share, comment, and collaborate on the same document with multiple people:

* The **SHARE** button (top-right of the toolbar) allows you to share the notebook and control permissions set on it.

* **File->Make a Copy** creates a copy of the notebook in Drive.

* **File->Save** saves the File to Drive. **File->Save and checkpoint** pins the version so it doesn't get deleted from the revision history. 

* **File->Revision history** shows the notebook's revision history. 

### (Extra) Commenting on a cell
You can comment on a Colaboratory notebook like you would on a Google Document. Comments are attached to cells, and are displayed next to the cell they refer to. If you have **comment-only** permissions, you will see a comment button on the top right of the cell when you hover over it.

If you have edit or comment permissions you can comment on a cell in one of three ways: 

1. Select a cell and click the comment button in the toolbar above the top-right corner of the cell.
1. Right click a text cell and select **Add a comment** from the context menu.
3. Use the shortcut **Ctrl+Shift+M** to add a comment to the currently selected cell. 

You can resolve and reply to comments, and you can target comments to specific collaborators by typing *+[email address]* (e.g., `+user@domain.com`). Addressed collaborators will be emailed. 

The Comment button in the top-right corner of the page shows all comments attached to the notebook.

## Section - Introducing Explorationlib

### Install _explorationlib_
Colab's come with many of the libraries we will need. It does not come with _explorationlib_. It's a module we will be using, and was written by Erik Peterson to support this course. Let's Install it.

In [ ]:
!pip install --upgrade git+https://github.com/parenthetical-e/explorationlib
!pip install --upgrade git+https://github.com/MattChanTK/gym-maze.git
!pip install celluloid # for the gifs

### Import modules

In [ ]:
# from the standard library
import shutil
import glob
import os
import copy
import sys

# these are common to scientific programming in python
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Computational experiments are run with 'experiment'
from explorationlib.run import experiment

# Here are some tools to select, save, and load
# data from computational experiments
from explorationlib.util import select_exp
from explorationlib.util import load
from explorationlib.util import save

# All the explorers we will play with are called
# "agents"; a bit of computer science jargon
from explorationlib.agent import Diffusion2d
from explorationlib.agent import Levy2d
from explorationlib.agent import TruncatedLevy2d

# The environments we will simulate live in a "gym"
from explorationlib.local_gym import Field
from explorationlib.local_gym import Bounded
from explorationlib.local_gym import Grid 
from explorationlib.local_gym import uniform_targets
from explorationlib.local_gym import constant_values

# A bunch of tools for plotting and for
# movie making
from explorationlib.plot import plot_position2d
from explorationlib.plot import plot_length_hist
from explorationlib.plot import plot_length
from explorationlib.plot import plot_targets2d
from explorationlib.plot import show_gif

# A couple metrics for scoring how well, or poorly,
# an exploration experiment went.
from explorationlib.score import search_efficiency
from explorationlib.score import total_reward
from explorationlib.score import first_reward

### Configure plots 
to be be nicer looking. We don't _have_ to do this, but _why not_?

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True

plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = "16"

### Better autocomplete and development
In our notebooks. Again, optional. If this errors out, and it might, skip it.

In [ ]:
%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2

## Section - getting to know our explorers

Run a diffusion and levy walker for 10000 steps, in an unbounded Field. Do 10 different experiments.

### Experimental Parameters

In [ ]:
# Setup
num_steps = 10000       # how many exploration steps in space?
num_experiments = 10    # we want to run ten experiments
detection_radius = 1

num_targets = 500

min_length = 0.1
step_size = 0.1  # Should be < (2 * detection_radius)

### Create Environment

In [ ]:
# Create env as an empty field
env = Field()

### Add targets to the environment

In [ ]:
# Targets
target_boundary = (50, 50)
targets = uniform_targets(num_targets, target_boundary)
values = constant_values(targets, 1)
env.add_targets(targets, values, detection_radius=detection_radius)

### Create Agents

In [ ]:
# Create agents
diffusion = Diffusion2d(
    min_length=min_length, 
    scale=0.1, 
    detection_radius=detection_radius, 
    step_size=step_size,
)

levy = Levy2d(min_length=min_length, 
    exponent=2, 
    detection_radius=detection_radius, 
    step_size=step_size,
)

### Run Experiments

In [ ]:
# experiments
levy_exp = experiment(
    f"levy_1", 
    levy, 
    env, 
    num_steps=num_steps, 
    num_experiments=num_experiments,
    seed=59393,
    dump=False,
)
diffusion_exp = experiment(
    f"diffusion_1", 
    diffusion, 
    env, 
    num_steps=num_steps, 
    num_experiments=num_experiments,
    seed=59393,
    dump=False,
)

### Plot the trajectories of both agents across the 10 experiments

In [ ]:
# View size
plot_boundary = (20, 20)

# Generate 10 plots of walking
for n in range(num_experiments):
    ax = None
    ax = plot_position2d(
        select_exp(levy_exp, n),
        boundary=plot_boundary,
        label=f"Levy",
        color="purple",
        alpha=0.6,
        ax=ax,
    )
    ax = plot_position2d(
        select_exp(diffusion_exp, n),
        boundary=plot_boundary,
        label=f"Diffusion",
        title=f"Experiment {n}",
        color="brown",
        alpha=0.6,
        ax=ax,
    )

### Question 1.1
Based on these 10 example experiments please describe how similar or different the Levy and diffusion search strategies seem to be? Visually, that is. Can you tell the difference between them in _all_ 10 examples?

_Note_: Don't be afraid to change the _plot boundary_ to zoom in or out. 

In [ ]:
# Write your answer here, as a python comment

### Plot the distribution of step sizes

Create one distribution for all Levy walks, and one distribution and plot them together. First plot the distributions, the plot them in log-log space. AKA the space in which power laws, like the Levy $p(l) = 1 / l^{\mu}$, are easiest to identify. 

In [ ]:
num_experiment = 0
ax = None
ax = plot_length_hist(
    select_exp(levy_exp, num_experiment),
    loglog=False,
    bins=60,
    density=True,
    alpha=0.6,
    label="Levy",
    color="purple",
    ax=ax,
    figsize=(6,4),
)
ax = plot_length_hist(
    select_exp(diffusion_exp, num_experiment),
    loglog=False,
    bins=50,
    density=True,
    alpha=0.6,
    color="brown",
    label="Diffusion",
    ax=ax,
)
sns.despine() # Make pretty plot

- Wow, the power law is steep right!? Can you see the exponential nature of diffusion?

In [ ]:
ax = None
ax = plot_length_hist(
    select_exp(levy_exp, num_experiment),
    loglog=True,
    bins=60,
    density=True,
    alpha=0.6,
    label="Levy",
    color="purple",
    ax=ax,
    figsize=(6,4),
)
ax = plot_length_hist(
    select_exp(diffusion_exp, num_experiment),
    loglog=True,
    bins=50,
    density=True,
    alpha=0.6,
    color="brown",
    label="Diffusion",
    ax=ax,
)
sns.despine() # Make pretty plot

### Compare the efficacies of the two strategies

Note: The difference between the agents might be smaller you expect, and the difference might not even be noticeable at all without a very large sample size.

Compile the scores of the experiments

In [ ]:
# Results, names, and colors
results = [levy_exp, diffusion_exp]
names = ["Levy", "Diffusion"]
colors = ["purple", "brown"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    r = search_efficiency(res)
    scores.append(r)  

Plot the distribution of scores

In [ ]:
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.5, bins=20)
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()

Plot the average score of each agent

In [ ]:
# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color="black", alpha=0.6)
plt.ylabel("Score")
plt.tight_layout()
sns.despine()